## Lab 9 Part 1: Semantic Segmentation

This notebook is about image segmentation.

We will perform augmentations using ```albumentations``` from [https://github.com/albumentations-team/albumentations](https://github.com/albumentations-team/albumentations). The advantage of this package in comparison with using pytorch augmentations is that it enables applying the same augmentation to an image and its segmentation mask. We will also use ```torchmetrics``` to obtain evaluation metrics. First, we need to install the required packages.

In [ ]:
!pip install -q -U albumentations
!echo "$(pip freeze | grep albumentations) is successfully installed"
!pip install torchmetrics

In [ ]:
import cv2
import os
from tqdm import tqdm
import random
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchsummary import summary
from torchmetrics import JaccardIndex
import pandas as pd

### Data Preparation

Let's start by downloading the data. We will be using the [Oxford-IIIT Pet Dataset](https://www.robots.ox.ac.uk/~vgg/data/pets/).

In [ ]:
!curl -L -O https://thor.robots.ox.ac.uk/~vgg/data/pets/images.tar.gz
!curl -L -O https://thor.robots.ox.ac.uk/~vgg/data/pets/annotations.tar.gz
!tar -xf images.tar.gz
!tar -xf annotations.tar.gz

The data was downloaded into the notebook. You can see it in Files (to the left of the notebook) in Google Colab. The images are in the folder ```images``` and the masks in ```annotations/trimaps```.

In the annotations you will notice a file named ```trainval.txt``` which contains the names of images typically used for training and validation, and ```test.txt``` which contains the images typically used for testing. Let's separate the data into training, validation and testing based on these files.

For today's class, we will only be using a part of the images during training (2000 for training, 500 for validation), to reduce the amount of time needed to run the notebook.

In [ ]:
images_directory = "images"
masks_directory = os.path.join("annotations", "trimaps")
trainval_filename = os.path.join("annotations", "trainval.txt")
test_filename = os.path.join("annotations", "test.txt")

# Obtain names of images for training and validation
trainval_file = pd.read_csv(trainval_filename, sep="\s+", header=None)
trainval_images_filenames = np.asarray(trainval_file[0].values)
trainval_images_filenames = np.asarray([x + ".jpg" for x in trainval_images_filenames])

# Shuffle images
random.seed(42)
random.shuffle(trainval_images_filenames)

# Split data into train and validation sets
train_images_filenames = trainval_images_filenames[:2000]
val_images_filenames = trainval_images_filenames[2000:2500]

# Obtain names of images for testing
test_file = pd.read_csv(test_filename, sep="\s+", header=None)
test_images_filenames = np.asarray(test_file[0].values)
test_images_filenames = np.asarray([x + ".jpg" for x in test_images_filenames])

This dataset has 3 labels, identified by integers from 1 to 3: pet (1), background (2), and border (3).

We can convert it to a binary problem by merging regions corresponding to pets and borders, obtaining binary masks with 2 labels: background (0) and pet/border (1).

In [ ]:
# Convert original mask into a binary mask
def preprocess_mask(mask):
    mask = mask.astype(np.float32)
    mask[mask == 2.0] = 0.0
    mask[(mask == 1.0) | (mask == 3.0)] = 1.0
    return mask

Visualize examples from the data

In [ ]:
def display_image_grid(images_filenames, images_directory, masks_directory):
    rows = len(images_filenames)
    _, ax = plt.subplots(nrows=rows, ncols=2, figsize=(10, 24))
    for i, image_filename in enumerate(images_filenames):
        image = cv2.imread(os.path.join(images_directory, image_filename))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        mask = cv2.imread(os.path.join(masks_directory, image_filename.replace(".jpg", ".png")), cv2.IMREAD_UNCHANGED,)
        mask = preprocess_mask(mask)
        ax[i, 0].imshow(image)
        ax[i, 1].imshow(mask, interpolation="nearest", cmap='gray')

        ax[i, 0].set_title("Image")
        ax[i, 1].set_title("Ground truth mask")

        ax[i, 0].set_axis_off()
        ax[i, 1].set_axis_off()

    plt.tight_layout()
    plt.show()

display_image_grid(train_images_filenames[:5], images_directory, masks_directory)

## Define Dataset

In [ ]:
class OxfordPetDataset(Dataset):
    def __init__(self, images_filenames, images_directory, masks_directory, transform=None):
        self.images_filenames = images_filenames
        self.images_directory = images_directory
        self.masks_directory = masks_directory
        self.transform = transform

    def __len__(self):
        return len(self.images_filenames)

    def __getitem__(self, idx):
        image_filename = self.images_filenames[idx]

        # Read image
        image = cv2.imread(os.path.join(self.images_directory, image_filename))

        # Convert from BGR to RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Normalize image into 0-1 range
        # Note that the ToTensorV2 method from the albumentations library does not automatically convert the image range into 0-1
        image = image / 255.

        # Read mask
        mask = cv2.imread(
            os.path.join(self.masks_directory, image_filename.replace(".jpg", ".png")), cv2.IMREAD_UNCHANGED,
        )

        # Preprocess mask by converting it into a binary mask
        mask = preprocess_mask(mask)

        # Apply the same data augmentation to both input image and target mask
        if self.transform is not None:
            transformed = self.transform(image=image, mask=mask)
            image = transformed["image"]
            mask = transformed["mask"]
        return image.float(), mask.to(torch.int64)

Define hyperparameters

In [ ]:
nr_classes = 2 # background vs foreground/pet
batch_size = 4
num_workers = 2
epochs = 10
learning_rate = 1e-4

# Get device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

Define transformations to be applied to the images and data loaders

In [ ]:
# Define transformations/augmentations to be applied to training data
train_transform = A.Compose(
    [
        A.Resize(256, 256),
        A.RandomCrop(224, 224),
        A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=30, p=0.5),
        ToTensorV2(),
    ]
)

# Define dataloader for training data
train_dataset = OxfordPetDataset(train_images_filenames, images_directory, masks_directory, transform=train_transform,)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True)

# Define transformations to be applied to validation data
val_transform = A.Compose(
    [
        A.Resize(256, 256),
        A.CenterCrop(224, 224),
        ToTensorV2()
    ]
)

# Define dataloader for validation data
val_dataset = OxfordPetDataset(val_images_filenames, images_directory, masks_directory, transform=val_transform,)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True)

## Model

We will use the segmentation network from [https://github.com/milesial/Pytorch-UNet](https://github.com/milesial/Pytorch-UNet).

In [ ]:
# Convolutional block - applies two convolutional layers in a row
class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)

# Downscaling block - downscales image and applies convolutional block
class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)

# Upscaling block - upscales image and applies convolutional block
class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


# Final convolutional block - applies convolutional layer
class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

# Segmentation network
class SegmentationNetwork(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=False):
        super(SegmentationNetwork, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        factor = 2 if bilinear else 1
        self.down4 = Down(512, 1024 // factor)
        self.up1 = Up(1024, 512 // factor, bilinear)
        self.up2 = Up(512, 256 // factor, bilinear)
        self.up3 = Up(256, 128 // factor, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

## Train the model

Start by defining the model, the optimizer, loss and metric to evaluate the model.

In this case, we will use the Jaccard Index (also known as intersection over union) as the metric. Since pytorch does not have this metric implemented, we will use the ```torchmetrics``` package. [Click here](https://lightning.ai/docs/torchmetrics/stable/) to find out more about the metrics available on torchmetrics.

In [ ]:
# Create model
model = # TODO
print(model)

# Put model in GPU
model.to(device)

# Define optimizer
optimizer = # TODO

# Define loss (crossentropy)
loss_fn = # TODO

# Define metric (e.g. Jaccard Index, Dice Coefficient, Pixel Accuracy)
metric = # TODO

Now implement the training cycle corresponding to one epoch.

In [ ]:
def epoch_iter(dataloader, model, loss_fn, optimizer=None, is_train=True):
    # TODO

Train the model for 10 epochs. Do not forget to use the validation set to save the best model at each epoch.

In [ ]:
# TODO

Plot metrics and loss on training and validation sets obtained during the training process

In [ ]:
def plotTrainingHistory(train_history, val_history):
    plt.subplot(2, 1, 1)
    plt.title('Cross Entropy Loss')
    plt.plot(train_history['loss'], label='train')
    plt.plot(val_history['loss'], label='val')
    plt.legend(loc='best')

    plt.subplot(2, 1, 2)
    plt.title('Jaccard Index')
    plt.plot(train_history['jaccard'], label='train')
    plt.plot(val_history['jaccard'], label='val')

    plt.tight_layout()
    plt.legend(loc='best')
    plt.show()

In [ ]:
plotTrainingHistory(train_history, val_history)

## Test the model

Visualize the results on only 10 images

In [ ]:
# Load the best model
# TODO

# Define test dataloader that loads only 10 images
inference_transform = # TODO
inference_dataset = # TODO
inference_dataloader = # TODO

# Visualize the results
with torch.no_grad():
    for img, mask in inference_dataloader:
        img, mask = img.to(device), mask.to(device)
        
        # Get prediction
        out = model(img)
        probs = F.softmax(out, dim=1)
        pred = torch.argmax(probs, dim=1)

        # Show original image, mask and prediction
        fig, ax = plt.subplots(ncols=3)
        ax[0].imshow(img[0].cpu().numpy().transpose((1, 2, 0)))
        ax[1].imshow(mask[0].cpu().numpy(), interpolation="nearest", cmap='gray')
        ax[2].imshow(pred[0].cpu().numpy(), interpolation="nearest", cmap='gray')

        ax[0].set_title("Image")
        ax[1].set_title("Ground truth mask")
        ax[2].set_title("Predicted mask")

        ax[0].set_axis_off()
        ax[1].set_axis_off()
        ax[2].set_axis_off()

        plt.tight_layout()
        plt.show()
